In [1]:
#sales org = 8000 (pharma & med surg) can only order controls
#add exception that chains must have 170 CSMP eligible customers (TOPCO chain)
#look into old/new DEA table Rakesh provided (remove the old DEA# from list)
#"CSMP_THRESHOLD"."PK_CSMP_THRESHOLD.TB::TA_CSMP_DEA_MAPPING" (DEA change history logged here)
#prioritize account if Business Type = 30 and assign as central fill
#update to ignore Business Type for Costco and Thrifty White's (Shep will manually update exceptions for customer segmentation)
#find out original Regulatory Channel logic (Bob Hartig & Anujit Sen) and apply to ZDEA batch file (only updates for new DEA)
#Customer segment INTRN replaced by MMS for dist chnl = 60 and chain = 969 --> 5/4/20

In [2]:
#ZDEA currently doesn't include Specialty customers (can be added but will need approval)

In [3]:
import pyodbc
cnxn = pyodbc.connect('DSN=BHP') 

#if connection is not working go to Control Panel --> ODBC Data Source --> System DSN --> reconfigure BHP esshanap10:38015 or esshanap20:38015
#update 6/25/19: change System DSN to BHPHANA:38015

In [4]:
import pandas as pd
import datetime
pd.options.display.max_columns = None
#5/16/19: added ADDED C.CUST_TYPE_BUS as ACT_CUST_TYPE and B.PMDKATR1 as ACT_BUS_TYPE
query="""SELECT A.EPKUNNR as ZDEA,A.SORT1 as DEA,A.NAME1 as ZDEA_Name,C.ACCT_DLVRY_ADDR as ACT_DLVRY_ADR,
C.ACCT_DLVRY_CTY_NAM as ACT_DLVRY_CTY,C.ACCT_DLVRY_ST_ABRV as ACT_DLVRY_ST,C.ACCT_DLVRY_ZIP as ACT_DLVRY_ZIP,
A.EXPIRY_DATE as ZDEA_Exp_Date,A.DEA_LIC_STATUS as ZDEA_LIC_STATUS,A.BUS_ACT_CODE as ZDEA_BUS_ACT_CODE,
A.DEA_CODE as ZDEA_Code,D.ZPHSDEACD_DESC as ZDEA_Code_Desc,A.DEA_REG_DC as ZDEA_REG_DC,A.DEA_CHAIN as ZDEA_CHAIN,
E.ZPHSDCHAI_DESC as CHAIN_DESC,A.DEA_REG_BUS_TYPE as ZDEA_REG_BUS_TYPE,A.DEA_STARTUP as ZDEA_STARTUP_PHCY,
A.DEA_ON_BOARDING as ZDEA_ON_BOARD,A.REG_STAT as ZDEA_REG_STATUS,A.REG_CHNL as ZDEA_REG_CHNL,A.THRESH_SEG as ZDEA_THRSH_SEG,
A.REG_SEGMENT as ZDEA_REG_SEG,A.REG_PROFILE as ZDEA_REG_PROF,B.CUSTOMER as ACCOUNT,B.CUSTOMER_NAME as ACT_NAME,
C.HOME_DC_ID as ACT_HOME_DC,B.ZCUSTCHID as ACT_CHAIN,C.CUST_GRP_NAM as ACT_CHAIN_DESC,B.ZCNLTGPC as ACT_NAT_GRP,
B.ZCNTLSGPC as ACT_SUB_GRP,B.CUST_CLASS,B.PMDKATR1 as ACT_BUS_TYPE,C.CUST_DIST_CHNL_TYP as ACT_DIST_CHNL, C.MCK_CO_CD,
B.ZDEAFMLY as ACT_DEA_FMLY,B.ACCOUNT_CLASSIFICATION as ACT_CLASSIFICATION,C.ACTIVE_CUST_IND,C.DEA_NUM_EXPIR_DT as ACT_DEA_EXP,
C.CUST_TYPE_BUS as ACT_CUST_TYPE_DESC,F.TXTMD as ACT_BUS_TYPE_DESC,G.CUST_TYP_CD, H.SALESORG, H.Division, 
H.DEL_INDIC, H.OBJVERS, H."/BIC/PMDAUFSD" as ORDER_BLOCK, C.DLVRY_RTE_NUM, C.DLVRY_RTE_STOP_NUM \
    FROM "_SYS_BIC"."PK_CSMP_DEAMASTER/CA_DEA_CSMP" A \
        LEFT JOIN "_SYS_BIC"."PK_CSMP_THRESHOLD.DM.MD/CA_CSMP_CUST_DEA" B on B.ZDEANUM = A.SORT1 \
        LEFT JOIN "_SYS_BIC"."PK_BUS_SEMANTIC.MD/C_CUSTOMER_MASTER" C on C.CUST_ACCT_ID = B.ZPRCSTN \
        LEFT JOIN "_SYS_BIC"."PK_CSMP_DEAMASTER/CA_DEA_CSMP_ZPHSDEACD" D on D.ZPHSDEACD = A.DEA_CODE \
        LEFT JOIN "_SYS_BIC"."PK_CSMP_DEAMASTER/CA_DEA_CSMP_ZPHSDCHAI" E on E.ZPHSDCHAI = A.DEA_CHAIN \
        LEFT JOIN "SAPBIW"."/BIC/TPMDKATR1" F on F."/BIC/PMDKATR1" = B.PMDKATR1 \
        LEFT JOIN "_SYS_BIC"."PK_BUS_SEMANTIC.MD/T_VSTX_CUST_TYPE" G on G.SAP_CUST_NUM = B.CUSTOMER \
        LEFT JOIN "SAPBIW"."/BI0/PCUST_SALES" H ON H.CUST_SALES = B.CUSTOMER \
    WHERE H.SALESORG = '8000' \
        AND H.DIVISION = '10' \
        AND H.PLANT <> '' \
        AND H.OBJVERS = 'A' \
    ORDER BY DEA, ACCOUNT"""

#consider joining "IW_IIS_STG"."PK_MCKESSON.MD.CUST::TA_PHMMMD_VSTX_CUST_ATTR_ALL" where XFD_KU_ORDBLKSLS_AR = ''

df=pd.read_sql(query,cnxn)


In [5]:
import numpy as np
cols = ['ZDEA_CHAIN','ACT_CHAIN','ACT_DIST_CHNL','ZDEA_REG_DC','ZDEA_REG_BUS_TYPE','ACT_HOME_DC','ACT_CLASSIFICATION']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df['Regulatory_Business_Type'] = ''
df['Regulatory_Segment'] = ''
df['Regulatory_Profile'] = ''
df['HospSegPriority'] = 1

df = df.replace(np.nan, '', regex=True) ##added 9/25

In [6]:
eligible_df = df[~df['ACT_DEA_FMLY'].isin(['INEL','#','TERM','NOAL','SUSP',''])]
#added code below test 3/28/19
#eligible_df['Regulatory_Segment'] = ''

eligible_df = eligible_df[(eligible_df.ACTIVE_CUST_IND != 'I')]
eligible_df = eligible_df[~eligible_df['ACT_DIST_CHNL'].isin([10])] # exclude non-order/billing accounts
eligible_df = eligible_df[eligible_df['CUST_TYP_CD'].isin(['DC02','DC03','PH01','PHML','SP02'])] # only include ordering accounts
eligible_df = eligible_df[~eligible_df['ACT_CHAIN'].isin(['0000000989'])] #exclude MPB (McK chains = 160 for intercompany but do order controls)
eligible_df = eligible_df[eligible_df['ACCOUNT'].notnull()]
#exclude order block and del indicator added 9/27/19
eligible_df = eligible_df[(eligible_df.ORDER_BLOCK == '')] 
eligible_df = eligible_df[(eligible_df.DEL_INDIC == '')]


today = datetime.datetime.today().strftime('%Y%m%d')
eligible_df = eligible_df[eligible_df.ACT_DEA_EXP >= today]
eligible_df['Priority'] = eligible_df.ACT_DIST_CHNL.map({40:10,30:11,20:12,50:13,60:14,70:15})
eligible_df.loc[eligible_df.ACT_NAME.str.contains("OTC|PHS|HBC"),'Priority'] = 19 #LOWER PRIORITY FOR HYVEE OTC/PHS/HBC ACCOUNTS
eligible_df.loc[eligible_df.ACT_HOME_DC == 8106,'Priority'] = 18 #LOWER PRIORITY FOR NRC HOME DC ACTS
eligible_df.loc[eligible_df.ACT_CLASSIFICATION == 4, 'Priority'] = 30 
#lower priority for MPB acts (4/9/19)
eligible_df.loc[eligible_df.ACT_CHAIN == '989', 'Priority'] = 16
#lower priority for central fills (3/28/19)
eligible_df.loc[eligible_df.ACT_BUS_TYPE == '30', 'Priority'] = 1 + eligible_df.loc[eligible_df.ACT_BUS_TYPE == '30', 'Priority'] 
#lower priority for Test Pricing national groups - includes DNS 'do not ship' and MMS customers (4/12/19)
eligible_df.loc[eligible_df.ACT_NAT_GRP == '9999', 'Priority'] = 16

eligible_df['Regulatory_Status'] = '01'

In [7]:
display = eligible_df
print(display)

              ZDEA        DEA                          ZDEA_NAME  \
0       0025015650  A90777889           405 86TH ST PHARMACY INC   
15      0025110114  A94143361            353 EMPIRE PHARMACY INC   
28      0025033795  AA0184921           CLINIC PRESCRIPTION SHOP   
37      0025004615  AA0347927                       ANMED HEALTH   
39      0025004615  AA0347927                       ANMED HEALTH   
...            ...        ...                                ...   
281135  0025009768  RY0468985  YUN, CHANGHYON CHRISTOPHER, PH.D.   
281137  0025142609  RY0482149                YU, SHAN PING (PHD)   
281139  0025108587  RY0552338                        YU, BINGLAN   
281140  0025115040  RZ0413550             ZHUO, JIA L. (MD, PHD)   
281142  0025097951  RZ0454746                        ZHANG, MING   

                    ACT_DLVRY_ADR ACT_DLVRY_CTY ACT_DLVRY_ST ACT_DLVRY_ZIP  \
0                 405 86TH STREET      BROOKLYN           NY         11209   
15                  505 UNI

In [8]:
ineligible_df = df[~df.index.isin(eligible_df.index)] 
ineligible_df['Priority'] = ineligible_df.ACT_DIST_CHNL.map({40:90,30:91,20:92,50:93,60:94,70:95,10:96})
ineligible_df.loc[ineligible_df.ACT_NAME.str.contains("OTC|PHS|HBC"),'Priority'] = 99 #LOWER PRIORITY FOR HYVEE OTC/PHS/HBC ACCOUNTS
ineligible_df.loc[ineligible_df.ACT_HOME_DC == 8106,'Priority'] = 98 #LOWER PRIORITY FOR NRC HOME DC ACTS
ineligible_df.loc[ineligible_df.ACT_CLASSIFICATION == 4, 'Priority'] = 100 
ineligible_df.loc[ineligible_df.ACT_BUS_TYPE == '30', 'Priority'] = 39
#lower priority for MPB acts (4/9/19)
ineligible_df.loc[ineligible_df.ACT_CHAIN == '989', 'Priority'] = 99
#lower priority for Test Pricing national groups - includes DNS 'do not ship' and MMS customers (4/12/19)
ineligible_df.loc[ineligible_df.ACT_NAT_GRP == '9999', 'Priority'] = 99
ineligible_df['Regulatory_Status'] = '05'

<ipython-input-8-2248cf287873>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ineligible_df['Priority'] = ineligible_df.ACT_DIST_CHNL.map({40:90,30:91,20:92,50:93,60:94,70:95,10:96})
C:\Users\eoshkdc\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\eoshkdc\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [9]:
status_df = eligible_df.append(ineligible_df, ignore_index=True)
status_df['Regulatory_Segment'] = status_df.ACT_DIST_CHNL.map({10:'MCK',20:'MHS',30:'ISMC',40:'RNA',50:'GOVT',60:'INTRN',70:'OTHER'})
status_df.loc[status_df.ZDEA_NAME.str.contains("MCKESSON"), 'Regulatory_Channel'] = 'MCK'
status_df['Regulatory_DC'] = status_df['ACT_HOME_DC'].fillna(status_df['ZDEA_REG_DC'])
status_df.loc[status_df.ZDEA_NAME.str.contains("MCKESSON CORPORATION"), 'Regulatory_Segment'] = 'OTHER'

##status_df = status_df.sort_values(by=['Priority']) moved to final step

In [10]:
## Exceptions for Chains
chains_df = status_df
chains_df['Regulatory_Chain_ID'] = ''
chains_df.loc[chains_df['ACT_CHAIN'].isin(['184','334','426','693','889','927','969']), 'Regulatory_Segment'] = 'OTHER'
chains_df.loc[chains_df['ACT_CHAIN'].isin(['385','41']), 'Regulatory_Segment'] = 'ISMC'

chains_df.loc[chains_df['ACT_CHAIN'].isin(['676']), 'Regulatory_Chain_ID'] = '021' #Albertsons Safeway
chains_df.loc[chains_df['ACT_CHAIN'].isin(['647']), 'Regulatory_Chain_ID'] = '023' #CVS HEALTH LTC
chains_df.loc[chains_df['ACT_CHAIN'].isin(['815']), 'Regulatory_Chain_ID'] = '022' #CVS HEALTH DSD
chains_df.loc[chains_df['ACT_CHAIN'].isin(['945']), 'Regulatory_Chain_ID'] = '024' #CVS HEALTH SPEC
chains_df.loc[chains_df['ACT_CHAIN'].isin(['766']), 'Regulatory_Chain_ID'] = '013' #KINNEY --> assign to separate segment for testing per Nate 2/8/19

chains_df.loc[(chains_df['ACT_CHAIN'].isin(['712'])) & (chains_df['ACT_NAT_GRP'].isin(['0667'])) & (chains_df['ACT_SUB_GRP'].isin(['000040'])), 'Regulatory_Chain_ID'] = '027' #TOPCO HY-VEE RX 
chains_df.loc[(chains_df['ACT_CHAIN'].isin(['712'])) & (chains_df['ACT_NAT_GRP'].isin(['0667'])) & (chains_df['ACT_SUB_GRP'].isin(['000240'])), 'Regulatory_Chain_ID'] = '028' #TOPCO WAKEFERN RX

chains_df.loc[chains_df['DEA'].isin(['RS0409424']), 'Regulatory_Segment'] = 'OTHER' #SAFECOR HEALTH, LLC

chains_df.loc[chains_df['DEA'].isin(['BC9645942']), 'Regulatory_Segment'] = 'RNA' #COSTCO CENTRAL FILL 583
chains_df.loc[chains_df['DEA'].isin(['BC9645942']), 'Regulatory_Chain_ID'] = '006' #COSTCO CENTRAL FILL 583

chains_df.loc[(chains_df['ACT_CHAIN'].isin(['659'])) & (chains_df['ACT_NAT_GRP'].isin(['3000'])) & (chains_df['ACT_SUB_GRP'].isin(['000020'])), 'Regulatory_Chain_ID'] = '659' #OptumRx Infusion

## Exceptions for MMS Regulatory Segment 5/4/2020 per Joel Perales
##chains_df.loc[(chains_df['ACT_CHAIN'].isin([969])) & (chains_df['Regulatory_Segment'].isin(['INTRN'])), 'Regulatory_Segment'] = 'MMS'
## Updated on 3/10/2021 per Joel Perales - Distribution Channel 60 no longer needed
chains_df.loc[(chains_df['ACT_CHAIN'].isin([969])), 'Regulatory_Segment'] = 'MMS'


##notes for chain name assignments
#Topco chains:
#- Chain name - Topco (e.g. Amber - Topco)
#
#TBA chains ignore b/c RNA already accounted for in first tab
#
#add new field in ZDEA for account name (e.g.BC6161474)
#add new field in ZDEA for IRP?
#
#chains w/ IRP:
#Associated Food Stores (90)
#SpartanNash (500, 510)
#Thrifty White (048)

In [11]:
## Exceptions for Govt customers
govt_df = chains_df
## added ACT_NAT_GRP 529 (DOD Mail Order) on 5/22/19
govt_df.loc[govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666','0529']), 'Regulatory_Segment'] = 'GOVT'

govt_df.loc[(govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666','0529'])) & (govt_df['ACT_SUB_GRP'].isin(['000001'])), 'Regulatory_Business_Type'] = '60' #VA 
govt_df.loc[(govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666','0529'])) & (govt_df['ACT_SUB_GRP'].isin(['000002'])), 'Regulatory_Business_Type'] = '61' #BOP
govt_df.loc[(govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666','0529'])) & (govt_df['ACT_SUB_GRP'].isin(['000004'])), 'Regulatory_Business_Type'] = '62' #IHS
govt_df.loc[(govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666','0529'])) & (govt_df['ACT_SUB_GRP'].isin(['000009'])), 'Regulatory_Business_Type'] = '63' #CMOP
govt_df.loc[(govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666','0529'])) & (~govt_df['ACT_SUB_GRP'].isin(['000001','000002','000004','000009'])), 'Regulatory_Business_Type'] = '64' #OTHER

In [12]:
## Exceptions to keep original ZDEA assignment(except Reg Status) if no account currently exists
#govt_df['Regulatory_DC'] = govt_df['Regulatory_DC'].fillna(govt_df['ZDEA_REG_DC'])
#govt_df['Regulatory_Channel'] = govt_df['Regulatory_Channel'].fillna(govt_df['ZDEA_REG_CHNL'])
#govt_df['Regulatory_Business_Type'] = govt_df['Regulatory_Business_Type'].fillna(govt_df['ZDEA_REG_BUS_TYPE'])
#govt_df['Regulatory_Chain_ID'] = govt_df['Regulatory_Chain_ID'].fillna(govt_df['ZDEA_CHAIN'])

In [13]:
## Exceptions to keep original ZDEA assignment(except Reg Status) if no account currently exists
govt_df.loc[govt_df.Regulatory_DC == '','Regulatory_DC'] = govt_df['ZDEA_REG_DC']
govt_df.loc[govt_df.Regulatory_Channel == '','Regulatory_Channel'] = govt_df['ZDEA_REG_CHNL']
govt_df.loc[govt_df.Regulatory_Business_Type == '', 'Regulatory_Business_Type'] = govt_df['ZDEA_REG_BUS_TYPE']
govt_df.loc[govt_df.Regulatory_Chain_ID == '', 'Regulatory_Chain_ID'] = govt_df['ZDEA_CHAIN']

##added for testing 3/28/19
govt_df.loc[govt_df.Regulatory_Segment == '', 'Regulatory_Segment'] = govt_df['ZDEA_REG_SEG']

In [14]:
## Exceptions for Regulatory Channel threshold calculations (based on Doug Hartig's original logic)
## need to reassign the logic based on the Bus Act Code and DEA Code instead
govt_df['Regulatory_Channel'] = 'OTHER'
govt_df.loc[govt_df['ZDEA_BUS_ACT_CODE'].isin(['B']), 'Regulatory_Channel'] = 'HOSP'
govt_df.loc[(govt_df['ZDEA_BUS_ACT_CODE'].isin(['C','M'])) & (~govt_df['ZDEA_CODE'].isin(['M2'])), 'Regulatory_Channel'] = 'PRACT'
govt_df.loc[govt_df['ZDEA_CODE'].isin(['A0','A3','A5','A7','A8']), 'Regulatory_Channel'] = 'RETL'

## Exceptions for Pure 340B 
## Added account classification 5 on 1/12/2022
govt_df.loc[govt_df['ACT_CLASSIFICATION'].isin([4,5]), 'Regulatory_Segment'] = '340B'

In [15]:
## Threshold Semgent field assignments (for threshold model customer segments)
govt_df.loc[govt_df['ZDEA_BUS_ACT_CODE'].isin(['A']), 'Threshold_Segmentation'] = '020' #PHARMACY

govt_df.loc[govt_df['ACT_CHAIN'].isin([676]), 'Threshold_Segmentation'] = '001' #ALBERTSONS SAFEWAY
govt_df.loc[(govt_df['ACT_CHAIN'].isin([202])) & (govt_df['ZDEA_CHAIN'].isin([206])), 'Threshold_Segmentation'] = '002' #COSTCO (chain)
govt_df.loc[govt_df['ACT_CHAIN'].isin([815]), 'Threshold_Segmentation'] = '003' #CVS HEALTH DSD
govt_df.loc[govt_df['ACT_CHAIN'].isin([647]), 'Threshold_Segmentation'] = '004' #CVS HEALTH LTC
govt_df.loc[govt_df['ACT_CHAIN'].isin([945]), 'Threshold_Segmentation'] = '005' #CVS HEALTH SPEC
govt_df.loc[govt_df['ACT_CHAIN'].isin([766]), 'Threshold_Segmentation'] = '013' #KINNEY
govt_df.loc[govt_df['ACT_CHAIN'].isin([68]), 'Threshold_Segmentation'] = '015' #HEB GROCERY
#11/16/18 - Nate requested to have all Meijer & Meijer Specialty under the Meijer customer segment
govt_df.loc[(govt_df['ACT_CHAIN'].isin([953])) & (govt_df['ACT_SUB_GRP'].isin(['000001'])), 'Threshold_Segmentation'] = '018' #MEIJER
govt_df.loc[(govt_df['ACT_CHAIN'].isin([953])) & (govt_df['ACT_SUB_GRP'].isin(['000002','000009'])), 'Threshold_Segmentation'] = '018' #MEIJER SPECIALTY pt1
govt_df.loc[govt_df['ACT_CHAIN'].isin([197]), 'Threshold_Segmentation'] = '018' #MEIJER SPECIALTY pt2                                
govt_df.loc[govt_df['ACT_CHAIN'].isin([252]), 'Threshold_Segmentation'] = '021' #RITE AID
govt_df.loc[govt_df['ACT_CHAIN'].isin([444]), 'Threshold_Segmentation'] = '022' #SHOPKO
govt_df.loc[govt_df['ACT_CHAIN'].isin([929]), 'Threshold_Segmentation'] = '023' #SUPERVALU
govt_df.loc[(govt_df['ACT_CHAIN'].isin([48])) & (govt_df['ZDEA_CHAIN'].isin([212])), 'Threshold_Segmentation'] = '024' #THRIFTY WHITE (corporate)
govt_df.loc[(govt_df['ACT_CHAIN'].isin([712])) & (govt_df['ACT_NAT_GRP'].isin(['0667'])) & (govt_df['ACT_SUB_GRP'].isin(['000040','000100','000250','000251','000252','000253'])), 'Threshold_Segmentation'] = '025' #TOPCO HY-VEE RX 
govt_df.loc[(govt_df['ACT_CHAIN'].isin([712])) & (govt_df['ACT_NAT_GRP'].isin(['0667'])) & (govt_df['ACT_SUB_GRP'].isin(['000240'])), 'Threshold_Segmentation'] = '026' #TOPCO WAKEFERN RX
govt_df.loc[govt_df['ACT_CHAIN'].isin([227]), 'Threshold_Segmentation'] = '029' #WALMART
govt_df.loc[govt_df['ACT_CHAIN'].isin([227]) & (govt_df.ACT_NAME.str.contains("NH")), 'Threshold_Segmentation'] = '030' #WALMART NH
govt_df.loc[govt_df['ACT_CHAIN'].isin([321]), 'Threshold_Segmentation'] = '031' #WEGMAN
govt_df.loc[govt_df['ACT_CHAIN'].isin([771]), 'Threshold_Segmentation'] = '008' #INGLES (ASSIGNED AS INEL)  


##DUEDILIGENCE_Q50M: CO_Sale> 0 and ZPHSDBUCO in(A,B) and ( CO_sales> 50M or RX_Sales > 50M) KEEP ORIGINAL LOGIC FOR SALES
##DUEDILIGENCE_NOCS: CO_Sale <= 0 or no sales
govt_df.loc[~govt_df['ZDEA_BUS_ACT_CODE'].isin(['A','B']), 'Threshold_Segmentation'] = '012' #DUEDILIGENCE_NONCRH
govt_df.loc[(~govt_df['ZDEA_BUS_ACT_CODE'].isin(['A'])) & (govt_df['ZDEA_CODE'].isin(['B0','B1','B2'])), 'Threshold_Segmentation'] = '016' #HOSPITAL
govt_df.loc[(~govt_df['ZDEA_BUS_ACT_CODE'].isin(['A'])) & (govt_df['ZDEA_CODE'].isin(['B3'])), 'Threshold_Segmentation'] = '017' #HOSPITALFED
govt_df.loc[govt_df['DEA'].isin(['BI7566245']), 'Threshold_Segmentation'] = '009' #DUEDILIGENCE_INJW
govt_df.loc[(govt_df['ZDEA_BUS_ACT_CODE'].isin(['A','B'])) & (govt_df['CUST_CLASS'].isin(['51','26'])), 'Threshold_Segmentation'] = '010' #DUEDILIGENCE_MAOF
govt_df.loc[(govt_df['ZDEA_BUS_ACT_CODE'].isin(['A','B'])) & (govt_df['ZDEA_CODE'].isin(['A1','A6'])), 'Threshold_Segmentation'] = '006' #DUEDILIGENCE_CENFIL
govt_df.loc[govt_df['ACT_BUS_TYPE'].isin(['30']), 'Threshold_Segmentation'] = '006' #DUEDILIGENCE_CENFIL pt2
govt_df.loc[(govt_df['ZDEA_BUS_ACT_CODE'].isin(['A','B'])) & (govt_df['CUST_CLASS'].isin(['37'])), 'Threshold_Segmentation'] = '007' #DUEDILIGENCE_FCI

govt_df.loc[(~govt_df['ACT_CHAIN'].isin(['989'])) & (govt_df['ACT_NAT_GRP'].isin(['0350','0657','0666'])) & (govt_df['ACT_SUB_GRP'].isin(['000001'])), 'Threshold_Segmentation'] = '27' #VA 

govt_df.loc[(govt_df['ZDEA_BUS_ACT_CODE'].isin(['A','B'])) & (govt_df['Regulatory_Channel'].isin(['340B'])), 'Threshold_Segmentation'] = '014' #DUEDILIGENCE_Z340B 
govt_df.loc[govt_df['Regulatory_Status'].isin(['05']), 'Threshold_Segmentation'] = '008' #DUEDILIGENCE_INEL

In [16]:
govt_df['ZDEA_CODE_DESC'] = govt_df['ZDEA_CODE_DESC'].str.replace('â€“','-')

In [17]:
finalstatus_df = govt_df
finalstatus_df['Priority'].fillna(999, inplace=True)
finalstatus_df = finalstatus_df.sort_values(by=['Priority']) ##added 9/21
finalstatus_df.drop_duplicates(subset='DEA', keep="first",inplace=True)
finalstatus_df['Regulatory_Status'].astype('str')
finalstatus_df.to_csv('ZDEA_MASTER_MMS2.csv', index = False)

In [18]:
#create ZDEA batch file for only new changes
#updated on 7/30/19 for two separate batch files, '05' registrants were losing Reg DC and Reg Segment assignments

upload_df = finalstatus_df
upload_df = upload_df.replace(np.nan, '', regex=True) 
upload_df = upload_df[(upload_df['ZDEA_REG_STATUS'] != upload_df['Regulatory_Status']) | 
            (upload_df['ZDEA_REG_SEG'] != upload_df['Regulatory_Segment']) | 
            (upload_df['ZDEA_REG_DC'] != upload_df['Regulatory_DC'])]
            ##(upload_df['ZDEA_CHAIN'] != upload_df['Regulatory Chain ID']) |  ## Reg Chain ID currently used in Cust Segmentation""
            ##(upload_df['ZDEA_REG_BUS_TYPE'] != upload_df['Regulatory_Business_Type'])]
            ## create similar logic for ZDEA Customer Segmentation once available
#upload_df = upload_df[['ZDEA','DEA','ZDEA_NAME','Regulatory_Status', 'Regulatory_Segment','Regulatory_DC']]
#upload_df.to_csv('ZDEA_Batch.csv', index = False)

In [19]:
upload_CSMPEligible_df = upload_df.loc[upload_df['Regulatory_Status']=='01']
upload_CSMPEligible_df = upload_CSMPEligible_df[['ZDEA','DEA','ZDEA_NAME','Regulatory_Status', 'Regulatory_Segment','Regulatory_DC']]
upload_CSMPEligible_df.to_csv('ZDEA_CSMP_Eligible_Batch.csv', index = False)

In [20]:
upload_CSMPIneligible_df = upload_df.loc[upload_df['Regulatory_Status']=='05']
upload_CSMPIneligible_df = upload_CSMPIneligible_df[['ZDEA','DEA','ZDEA_NAME','Regulatory_Status']]
upload_CSMPIneligible_df.to_csv('ZDEA_CSMP_Inlegible_Batch.csv', index = False)

In [21]:
#create RNA Master File
rna_df = finalstatus_df
rna_df = rna_df[(rna_df.Regulatory_Segment == 'RNA')]
rna_df = rna_df[(rna_df.Regulatory_Status == '01')]
rna_df = rna_df[['DEA','ACT_NAME','Regulatory_Chain_ID','ACT_CHAIN','ACT_CHAIN_DESC','ACT_SUB_GRP','ZDEA_CODE','ZDEA_CODE_DESC','ZDEA_REG_DC','ACT_DLVRY_ADR','ACT_DLVRY_CTY','ACT_DLVRY_ST','ACT_DLVRY_ZIP',]]
rna_df.to_excel('ZDEA_RNA_MASTER.xlsx', index = False)
    
#add account chain name and act sub group name

In [22]:
#finalstatus_df.loc[finalstatus_df.ACT_NAME.str.contains("OTC|PHS"), 'Priority'] += .1 (lower priority instead of removing if possible) 
#finalstatus_df = finalstatus_df[finalstatus_df.ACT_NAME.str.contains("OTC|PHS")==False] # HYVEE accounts


In [23]:
#updated 8/1/19: adding act Cust/Bus Type values for hospital segmentation
Hospital_df = df[['DEA','ACCOUNT','ZDEA_CODE','ZDEA_CODE_DESC','ACT_CUST_TYPE_DESC','HospSegPriority']].copy()

#list of allowed Act_Cust_Types as provided by AGI 8/1/19
allowed_vals = ['HOSPITAL GROUP','HOSPITAL (INDIVIDUAL)','DRUG CHAIN','LARGE DRUG CHAIN','CLOSED PHARMACY','HOSPITAL CHAIN','EMERGENCY CARE CENTERS','UNIVERSITY HOSPITAL','CHAIN WAREHOUSE','GROCERY/DRUG COMBINATION','MAIL ORDER FACILITY','HMO PHARMACY (NON-HOSP)','WHOLESALER MCKESSON','SURGICENTERS','PHYSICIAN/DENTIST','HOME IV THERAPY','STATE HOSPITAL','NURSING HOME W/ PHARMACY','ONCOLOGY CENTERS','STATE CORRECT INSTITUTE','SUBSTANCE ABUSE CENTERS','DIALYSIS CENTERS','HMO WAREHOUSE','HOSPICES','MCKESSON DC','DME RETAILER','COUNTY CORRECT INSTITUTE','CITY CORRECT INSTITUTE','NON-ORDER CUSTOMER','MCKESSON ACCOUNT MANAGER','WHOLESALER NON-MCKESSON','NURSING HOME W/PHARMACY',' ','']
Hospital_df.loc[~Hospital_df["ACT_CUST_TYPE_DESC"].isin(allowed_vals), "ACT_CUST_TYPE_DESC"] = "Non-Hospital Segmentation"

Hospital_df.loc[Hospital_df.ACT_CUST_TYPE_DESC == "Non-Hospital Segmentation", 'HospSegPriority'] = 99

Hospital_df = Hospital_df[Hospital_df['ZDEA_CODE'].isin(['B0'])]
Hospital_df.rename(columns={'ACT_CUST_TYPE_DESC':'HOSP_SEG(ACT_CUST_TYPE)'}, inplace=True)
Hospital_df = Hospital_df[['DEA','HOSP_SEG(ACT_CUST_TYPE)','HospSegPriority']]

Hospital_df = Hospital_df.sort_values(by=['HospSegPriority'])
Hospital_df.drop_duplicates(subset='DEA', keep="first",inplace=True)

Hospital_FINAL_df = pd.merge(finalstatus_df, Hospital_df, on="DEA",how="left")
Hospital_FINAL_df.to_excel('HospitalSegmentTest.xlsx', index = False)

In [24]:
#Hospital_df = Hospital_df[(Hospital_df.DEA)]
##Hospital_df = Hospital_df[Hospital_df['ACT_CUST_TYPE_DESC'].isin(['HOSPITAL GROUP','HOSPITAL (INDIVIDUAL)','DRUG CHAIN','LARGE DRUG CHAIN','CLOSED PHARMACY','HOSPITAL CHAIN','EMERGENCY CARE CENTERS','UNIVERSITY HOSPITAL','CHAIN WAREHOUSE','GROCERY/DRUG COMBINATION','MAIL ORDER FACILITY','HMO PHARMACY (NON-HOSP)','WHOLESALER MCKESSON','SURGICENTERS','PHYSICIAN/DENTIST','HOME IV THERAPY','STATE HOSPITAL','NURSING HOME W/ PHARMACY','ONCOLOGY CENTERS','STATE CORRECT INSTITUTE','SUBSTANCE ABUSE CENTERS','DIALYSIS CENTERS','HMO WAREHOUSE','HOSPICES','MCKESSON DC','DME RETAILER','COUNTY CORRECT INSITUTE','CITY CORRECT INSTITUTE','NON-ORDER CUSTOMER','MCKESSON ACCOUNT MANAGER',' ',
##])]